In [2]:

import theano
import numpy
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.layers import SimpleRNN
from keras.layers import Flatten
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import cross_validate
import keras
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

In [3]:
#seeding for reproducibility i.e for same results for a given input every time the program is run
seed = 7
numpy.random.seed(seed)

In [4]:
theano.config.optimizer='fast_compile'
theano.config.exception_verbosity='high'
theano.config.compute_test_value = 'off'

In [11]:
#loading the dataset and creating numeric labels as keras only accepts numeric inputs and outputs.
positive_examples=list(open('C:\\Users\\Vineet\\Desktop\\positive.txt',mode='r',encoding='utf8'))
positive_examples = [s.strip() for s in positive_examples]
negative_examples=list(open('C:\\Users\\vineet\\Desktop\\Negative.txt',mode='r',encoding='utf8'))
negative_examples = [s.strip() for s in negative_examples]
x=positive_examples+negative_examples
#print(x)


In [12]:
xa=np.asarray(x)
print (xa.shape)
positive_labels = [[1] for _ in positive_examples]
negative_labels = [[0] for _ in negative_examples]
y = np.concatenate([positive_labels, negative_labels], 0)
print (y.shape)


(727,)
(727, 1)


In [35]:
#using train_test_split to automatically split data into train and test in ratio 70:30 respectively.
from sklearn.model_selection import train_test_split
xa_train,xa_test,y_train,y_test=train_test_split(xa,y,test_size=0.3,random_state=4)
print(xa_train.shape)
print(xa_test.shape)
print(y_train.shape)
print(y_test.shape)
print(xa_train[0])

(508,)
(219,)
(508, 1)
(219, 1)
भारत के महाकवि कालिदास के हाथों मारे गए थे :


In [20]:
#tokenizer used to split each line into words and then labelling each word with a key(a numeric value) 
#so as to generate an array to pass in furthur functions using pad_sequences.
tokenizer = Tokenizer(num_words=None,split=' ',lower=True)
tokenizer.fit_on_texts(xa_train)
integer_sentences_train = tokenizer.texts_to_sequences(xa_train)
data_train = pad_sequences(integer_sentences_train,padding='post',truncating='post',value=0.)
print(data_train[0])
top_words = 5000 #len(tokenizer.word_index)
print(top_words)
max_words = 30
print(xa_train[0])

[  23    2 1071 1072    2  329  431   81   51    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
5000
भारत के महाकवि कालिदास के हाथों मारे गए थे :


In [21]:
tokenizer.fit_on_texts(xa_test)
integer_sentences_test = tokenizer.texts_to_sequences(xa_test)
data_test = pad_sequences(integer_sentences_test,padding='post',truncating='post',value=0.)
print(data_test[0])
print(xa_test[0])

[  70 3467    8 3468  136 3469 3470 3471   28 3472 3473 3474   19 3475
   54 3476 3477  196    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
अपनी कमजोरियोँ का जिक्र कभी नकरना जमाने से। लोग कटी पतंगको जम कर लुटा करते हैँ॥ #MinionsOnStarMovies #KashmirFloods


In [22]:
data_train = sequence.pad_sequences(data_train, maxlen=max_words, dtype='float32')
data_test = sequence.pad_sequences(data_test, maxlen=max_words, dtype='float32')

In [23]:
#creating the model i.e adding the layers to the model
model = Sequential()
model.add(Embedding(input_dim = top_words, output_dim = 20, input_length=max_words))

model.add(Convolution1D(nb_filter=28, filter_length=3, border_mode='same', activation='tanh'))
model.add(MaxPooling1D(pool_length=2))
model.add(SimpleRNN(100,return_sequences=True))
model.add(LSTM(80,return_sequences=True))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

C:\Users\Vineet\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=28, kernel_size=3, padding="same")`
  """
C:\Users\Vineet\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 20)            100000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 28)            1708      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 28)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 15, 100)           12900     
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 80)            57920     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)               

In [48]:
#training and testing the model and generating accuracy score.
model.fit(data_train[:-1],y_train[:-1], batch_size=128,epochs=5,validation_data=(data_test, y_test), verbose=1,sample_weight=None, initial_epoch=0)
yp = model.predict(data_test[:-1], batch_size=32, verbose=1)
print(yp)
ypreds = np.argmax(yp, axis=1)
scores = model.evaluate(data_test[:-1], y_test[:-1], verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 507 samples, validate on 219 samples
Epoch 1/5
507/507 [==============================] - 0s 963us/step - loss: 0.1969 - accuracy: 0.9290 - val_loss: 0.8941 - val_accuracy: 0.6530
Epoch 2/5
507/507 [==============================] - 0s 934us/step - loss: 0.0650 - accuracy: 0.9803 - val_loss: 1.3355 - val_accuracy: 0.7215
Epoch 3/5
507/507 [==============================] - 0s 876us/step - loss: 0.0462 - accuracy: 0.9882 - val_loss: 1.9228 - val_accuracy: 0.6575
Epoch 4/5
507/507 [==============================] - 0s 930us/step - loss: 0.0239 - accuracy: 0.9941 - val_loss: 2.3371 - val_accuracy: 0.6256
Epoch 5/5
218/218 [==============================] - 0s 415us/step
[[3.58537375e-03]
 [1.33017853e-01]
 [2.66101386e-04]
 [9.90972016e-03]
 [1.04043036e-04]
 [6.12499595e-01]
 [1.11515292e-04]
 [5.92171855e-04]
 [1.28153088e-05]
 [9.95210826e-01]
 [2.22414755e-03]
 [5.41307636e-05]
 [9.94985819e-01]
 [9.24064636e-01]
 [5.68535971e-03]
 [1.21380240e-01]
 [7.18486845e-05]
 [6.66086

In [67]:
test_examples=list(open('C:\\Users\\vineet\\Desktop\\test_doc.txt',mode='r',encoding='utf8'))
test_examples = [s.strip() for s in test_examples]
test_examples_a=np.asarray(test_examples)
print (test_examples_a.shape)
print(test_examples_a[2])
tokenizer = Tokenizer(num_words=None,split=' ',lower=True)
tokenizer.fit_on_texts(test_examples_a)
integer_sentences_test_examples = tokenizer.texts_to_sequences(test_examples_a)
data_test_examples = pad_sequences(integer_sentences_test_examples,padding='post',truncating='post',value=0.,maxlen=30)
print(integer_sentences_test_examples[0])
print(data_test_examples[0])
top_words = 5000
max_words = 30


(3,)
हमारे पास पैसा नहीं है
[3, 4, 2, 5, 6, 1]
[3 4 2 5 6 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [68]:
data_test_examples = sequence.pad_sequences(data_test_examples, maxlen=max_words, dtype='float32')

In [69]:
y_test_examples = model.predict(data_test_examples, batch_size=32, verbose=1)

3/3 [==============================] - 0s 5ms/step


In [70]:
print(y_test_examples)

[[9.4233853e-01]
 [3.6825455e-04]
 [3.5723451e-01]]
